In [ ]:
import cv2 
import itertools
import numpy as np
from scipy.cluster.vq import kmeans,vq
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.cluster import KMeans
import operator
from functools import reduce

In [ ]:
def quantize_kmeans(a, b,ncolors, npics):
    print(len(a))
    print(1)
    lengthab = len(a) / npics
    
#     w,h = np.shape(a)
        
    # reshape matrix
    pixel = np.reshape((cv2.merge((a,b))),(len(a),2)).astype(float)
    
    # quantization
    kmeans = KMeans(n_clusters=ncolors) 
    label=kmeans.fit(pixel)
    return label.labels_

In [ ]:
def train_xgb(files,ncolors,npics):
    
    kmap_a=np.array([])
    kmap_b=np.array([])
    for f in files:

        _,a,b = load_image(f)
        kmap_a = np.concatenate([kmap_a, a.flatten()])
        kmap_b = np.concatenate([kmap_b, b.flatten()])

    label=quantize_kmeans(kmap_a, kmap_b, ncolors, npics)
    
    pixel=[]
    for f in files:
        l, a, b = load_image(f)
        this_pixel = l.reshape(1,256*256).tolist()
        #print(len(this_pixel[0]))
        pixel.append(this_pixel[0])

    pixel = reduce(operator.add, pixel)

    pixel_mat = np.array(pixel).reshape(npics*256*256,1)

    dtrain = xgb.DMatrix(pixel_mat, label=label)
    param = {'max_depth':10, 'eta':0.1, 'silent':1, 'objective':'multi:softmax', 'num_class':32}
    num_round = 10
    bst = xgb.train(param, dtrain, num_round)
    return bst

In [ ]:
l_test, a_test, b_test = load_image('C:/Users/rolco/Desktop/test2.jpg')
pixel_test = l_test.reshape(256*256,1)
#dtrain = xgb.DMatrix(pixel, label=label)
dtest=xgb.DMatrix(pixel_test)
preds = bst.predict(dtest)
#print(np.max(preds),np.min(preds))

In [ ]:
preds = preds.reshape(256,256)
preds.shape
colors_present = [i for i in range(16)]
output_a = np.zeros((256,256))
output_b = np.zeros((256,256))
output_l = l_test
for i in range(256):
    for j in range(256):
#             output_labels[i,j] = tmp[0]
#         print(colors_present[int(preds[i,j])],i,j)
        a,b = label_to_color_map_value[colors_present[int(preds[i,j])]]
        output_a[i,j] = a
        output_b[i,j] = b
        
output_img = cv2.cvtColor(cv2.merge(( np.uint8(output_l), np.uint8(output_a), np.uint8(output_b))), cv2.COLOR_LAB2RGB)
plt.imshow(output_img)
# plt.imsave('output_img1.jpg',output_img)